# 답변 생성(담당자 안내)

In [ ]:
# nodes/fallback.py
# 부서별 키워드 및 연락처 (상수로 정의)
HANDOFF_RULES = [
    (
        ["급여", "연봉", "인사", "채용", "입사", "퇴사", "휴가", "연차", "복리후생", "평가", "승진"],
        "인사",
        "hr@gaida.play.com",
        "슬랙 #ask-hr"
    ),
    (
        ["회계", "결제", "지출", "예산", "재무", "세금", "계산서", "청구서", "지급", "지출"],
        "재무",
        "fi@gaida.play.com",
        "슬랙 #ask-fi"
    ),
    (
        ["사무실", "비품", "물품", "구매", "수령", "우편", "문서", "서류", "출장", "숙박", "교통"],
        "총무",
        "ga@gaida.play.com",
        "슬랙 #ask-ga"
    ),
    (
        ["서버", "네트워크", "IT", "컴퓨터", "장비", "시스템", "접속", "인프라", "VPN", "계정", "접근"],
        "인프라",
        "in@gaida.play.com",
        "슬랙 #ask-in"
    ),
    (
        ["보안", "해킹", "정보", "유출", "침해", "랜섬웨어", "백신", "접근제어", "인증", "암호"],
        "보안",
        "se@gaida.play.com",
        "슬랙 #ask-se"
    )
]

def generate_handoff_response(state: GraphState) -> dict:
    question = state["original_question"].lower()  # 원본 질문 사용
    matched_dept = None
    
    # 부서별 키워드 체크 (위에서부터 순차 검색)
    for keywords, dept, email, slack in HANDOFF_RULES:
        if any(keyword in question for keyword in keywords):
            matched_dept = (dept, email, slack)
            break
    
    # 매칭된 부서가 없을 시 인사팀으로 기본 설정
    if matched_dept is None:
        dept, email, slack = "인사", "hr@gaida.play.com", "슬랙 #ask-hr"
    else:
        dept, email, slack = matched_dept
    
    # 응답 템플릿 (동적 값 삽입)
    response = f"""
    👋 안녕하세요. '{state['original_question']}'에 대한 문의는 **{dept}팀**에 하시면 정확한 도움을 드릴 수 있습니다.

    📧 이메일: {email}  
    💬 {slack}  

    감사합니다!
    """
    
    return {"answer": response}

In [ ]:
# nodes/handoff.py
from typing import Dict
from langgraph.graph import GraphState  # 공유 state 파일 import (팀과 공유)

# 부서 정보 dict (이메일과 슬랙 채널)
DEPARTMENTS = {
    "인사": {"email": "hr@gaida.play.com", "slack": "#ask-hr"},
    "재무": {"email": "fi@gaida.play.com", "slack": "#ask-fi"},
    "총무": {"email": "ga@gaida.play.com", "slack": "#ask-ga"},
    "인프라": {"email": "in@gaida.play.com", "slack": "#ask-in"},
    "보안": {"email": "se@gaida.play.com", "slack": "#ask-se"},
}

# 부서별 키워드 매핑 (확장 가능)
DEPARTMENT_KEYWORDS = {
    "인사": ["급여", "연봉", "휴가", "연차", "채용", "복지", "인사평가", "퇴직", "입사", "퇴사","평가", "승진"],
    "재무": ["결제", "세금", "예산", "회계", "지출", "송금", "계산서", "청구서", "지급", "비용", "환급"],
    "총무": ["사무실", "비품", "물품", "구매", "수령", "우편", "사무용품", "시설", "행사", "차량", "청소", "자산", "출장", "숙박", "교통"],
    "인프라": ["서버", "네트워크", "컴퓨터", "it", "소프트웨어", "인프라", "장비", "시스템", "접속", "vpn", "계정", "접근"],
    "보안": ["보안", "해킹", "정보", "유출", "침해", "랜섬웨어", "백신", "데이터", "비밀번호", "방화벽", "악성코드", "암호"],
}

def determine_department(question: str) -> str:
    """
    질문에 따라 적합한 부서를 결정. 매칭 안 되면 '인사'로 폴백.
    """
    question_lower = question.lower()  # 대소문자 무시
    
    for dept, keywords in DEPARTMENT_KEYWORDS.items():
        if any(keyword in question_lower for keyword in keywords):
            print(f"부서 매칭: {dept} (키워드 매칭)")
            return dept
    
    print("부서 매칭 실패: 인사팀으로 폴백")
    return "인사"  # 폴백

def generate_handoff_response(state: GraphState) -> Dict[str, str]:
    """
    담당자 안내 답변 생성 노드.
    - 질문에 따라 부서 결정 후, 맞춤 안내 메시지 생성.
    - state 업데이트: "answer" 필드에 안내 메시지 저장.
    """
    question = state["question"]  # state에서 질문 읽기
    
    # 부서 결정
    dept = determine_department(question)
    dept_info = DEPARTMENTS[dept]
    
    # 안내 메시지 생성 (템플릿 기반, 질문 요약 포함)
    answer = (
        f"안녕하세요! '{question}'에 대한 질문은 더 정확한 안내를 위해 "
        f"{dept}팀 담당자에게 직접 문의해 주세요.\n"
        f"- 이메일: {dept_info['email']}\n"
        f"- 슬랙: {dept_info['slack']}\n"
        "추가 질문이 있으시면 언제든 말씀해 주세요. 감사합니다!"
    )
    
    # (옵션) Level 2: LLM으로 더 자연스러운 메시지 생성
    # prompt = f"다음 질문에 대해 {dept}팀 안내 메시지를 자연스럽게 작성해. 연락처 포함: {question}"
    # answer = llm.invoke(prompt).content
    
    print(f"담당자 안내 생성: {dept}팀 - {answer}")
    return {"answer": answer, "route_decision": "handoff"}  # state 업데이트 (필요 시)

# 테스트 코드 (단독 테스트용)
if __name__ == "__main__":
    # 테스트 케이스 1: 인사 매칭
    mock_state = {"question": "연봉 인상 신청 어떻게 해?"}
    print(generate_handoff_response(mock_state)["answer"])
    # 예상 출력: 인사팀 안내 메시지

    # 테스트 케이스 2: 재무 매칭
    mock_state = {"question": "급여 세금 계산 방법 알려줘"}
    print(generate_handoff_response(mock_state)["answer"])
    # 예상 출력: 재무팀 안내 메시지

    # 테스트 케이스 3: 보안 매칭
    mock_state = {"question": "데이터 유출 방지 어떻게 해?"}
    print(generate_handoff_response(mock_state)["answer"])
    # 예상 출력: 보안팀 안내 메시지

    # 테스트 케이스 4: 매칭 실패 (폴백)
    mock_state = {"question": "날씨가 좋아요"}
    print(generate_handoff_response(mock_state)["answer"])
    # 예상 출력: 인사팀 안내 메시지